# Base de información tipo 15, al que se añade informanción relevante de tipos 11, y 14

tipo11: parcelas catastrales, coordenadas xy

tipo14: construcciones dentro de las unidades constructivas: año antiguedad, año reforma si hay, superficie, garage

tipo15: registro inmuebles. Tabla principal en la que poblar los datos relevantes de las otras tablas

In [2]:
### imports y definición de directorios
import pandas as pd
import os
import numpy as np
import csv

input_dir = '../../../data/clean/CAT/'
out_dir = '../../../data/clean/ficheros_preparados/'
processed_dir = '../../../data/clean/CAT/procesados/'

pending_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

pending_files = [f for f in pending_files if f[0:3] == 'T15']

for f in pending_files:
    
    f15 = os.path.join(input_dir, f)    
    
    ## cargamos fichero tipo 15 (fincas)

    indata15 = pd.read_csv(f15, low_memory=False, dtype='str')
    cols_eliminar = indata15.columns[indata15.columns.str.contains('blanc')]
    cols_eliminar = cols_eliminar.union(['tipo_reg', 'ctrl1', 'ctrl2', 'cmun_origen', 'cod_zona_concent', 'cod_poligono', 'cod_parcela', 'cod_paraje', 'nombre_paraje', 'num_orden_escrit_hori'])
    indata15 = indata15.drop(cols_eliminar, axis = 1)

    ## cargamos fichero tipo 14, para obtener m2 y años de construcción y reforma
    ## t15 y t14 se cruzan por (parcela, num_cargo) en t15 y (parcela, num_orden_BI) en t14. PUeden existir
    ## varios registros en t14 asociados a uno de 15. P. ej. viviendas con garajes. De aquí se puede marcar
    ## que viviendas tienen garaje incluido para luego estimar el coste

    f14 = f15.replace('T15', 'T14')
    indata14 = pd.read_csv(f14, low_memory=False, dtype='str')
    ## reducimos a sólo columnas de cruce y datos de interes
    cols_interes = ['parcela_cat', 'num_orden_BI', 'tipo_reforma', 'anio_ref', 'anio_antiguedad', 'local_interior', 'm2_total', 'm2_porches_terr', 'm2_imputables_loc', 'tipologia_constr', 'planta']
    indata14 = indata14.reindex(columns=cols_interes)

    ## muestra para acelerar desarrollo, comentar en producción
    #indata15 = indata15[0:200000]
    #indata14 = indata14[0:200000]

    ## dejamos en t14 sólo registros relacionados con t15 que sean viviendas, el resto no es de interés. Mejora
    ## rendimiento de groupby/transform posteriores
    indata15['parcela_cat'].fillna('No info', inplace = True)
    indata15['num_cargo'].fillna('No info', inplace = True)
    duplas_viv = indata15[indata15['clave_grupo_BI'] == 'V'].groupby(['parcela_cat', 'num_cargo'])
    duplas_viv = duplas_viv.size().reset_index()
    duplas_viv = duplas_viv.drop(0, axis = 1)
    indata14 = pd.merge(indata14, duplas_viv, left_on=['parcela_cat','num_orden_BI'], right_on=['parcela_cat', 'num_cargo'])


    ## seleccionamos las duplas de t14 con más de un registro y alguno en plantas negativas -> viviendas con garages
    indata14['parcela_cat'].fillna('No info', inplace = True)
    indata14['num_orden_BI'].fillna('No info', inplace = True)
    ## la línea de abajo es extremadamente lenta, preguntar pq. Sustituyo por group y merge
    #indata14['registros'] = indata14.groupby(['parcela_cat', 'num_orden_BI'])[['parcela_cat']].transform(lambda x: len(x))
    grupo_comp = indata14.groupby(['parcela_cat', 'num_orden_BI'])[['parcela_cat']].size().reset_index()
    indata14 = pd.merge(indata14, grupo_comp, how='left', left_on=['parcela_cat', 'num_orden_BI'], right_on=['parcela_cat', 'num_orden_BI'])
    indata14.rename(columns={0:'registros'}, inplace=True)
    indata14['planta'].fillna('No info', inplace = True)
    indata14['posible_garage'] = indata14.apply(lambda x: 1 if ((x['registros'] > 1) & (x['planta'][0] == '-')) else 0, axis=1)
    duplas14_garage = indata14[indata14['posible_garage'] == 1].groupby(['parcela_cat', 'num_orden_BI']).size().reset_index()
    duplas14_garage.drop(0, axis = 1, inplace=True)
    duplas14_garage['garage']=1

    ## marcamos en t15 las viviendas con garage
    indata15 = pd.merge(indata15, duplas14_garage, how='left', left_on=['parcela_cat', 'num_cargo'], right_on=['parcela_cat','num_orden_BI'])
    indata15['garage'].fillna(0, inplace = True)
    indata15.drop('num_orden_BI', axis = 1, inplace = True)
    indata15['garage'] = indata15['garage'].map(int)

    ## dejamos un unico valor de año, el maximo de la reforma y de antiguedad

    years = ['anio_ref', 'anio_antiguedad']
    indata14[years] = indata14[years].apply(lambda x: x.fillna(0))
    indata14[years] = indata14[years].applymap(int)
    indata14['aniot14'] = np.maximum(indata14['anio_ref'], indata14['anio_antiguedad'])
    duplas14_maxyear = indata14.groupby(['parcela_cat', 'num_orden_BI'])['aniot14'].max().reset_index()
    indata15 = pd.merge(indata15, duplas14_maxyear, how='left', left_on=['parcela_cat', 'num_cargo'], right_on=['parcela_cat','num_orden_BI'])
    years = ['antiguedad_BI', 'aniot14']
    indata15[years] = indata15[years].apply(lambda x: x.fillna(0))
    indata15[years] = indata15[years].applymap(int)
    indata15['anio_mejor'] = np.maximum(indata15['antiguedad_BI'], indata15['aniot14'])

    ## poblar con las coordenadas xy desde tipo 11 parcelas

    f11 = f15.replace('T15', 'T11')
    indata11 = pd.read_csv(f11, low_memory=False, dtype='str')
    ## Solo necesitamos las coordenadas y la proyección de esta tabla
    ## convertimos las coordenadas a float, los 2 ultimos digitos son el decimal
    cols_coor = indata11.columns[indata11.columns.str.contains('coor')]
    indata11[cols_coor] = indata11[cols_coor].apply(lambda x: x.fillna(0))
    indata11[cols_coor] = indata11[cols_coor].applymap(lambda x: int(x)/100.0)
    ## quitar 'EPSG:' de huso_geo
    indata11['huso_geo'].fillna('', inplace = True)
    indata11['huso_geo'] = indata11['huso_geo'].apply(lambda x: x[5:])

    indata15 = pd.merge(indata15, indata11[['parcela_cat', 'x_coor', 'y_coor', 'huso_geo']], how='left', left_on='parcela_cat', right_on='parcela_cat')

    ## grabamos el fichero de resultado 
    fbienesinmuebles = f.replace('T15', 'BI')
    fbienesinmuebles = fbienesinmuebles.replace('.gz', '')
    fbienesinmuebles = os.path.join(out_dir, fbienesinmuebles)

    indata15.to_csv(fbienesinmuebles, index=False, sep='^', quoting= csv.QUOTE_NONNUMERIC, encoding='utf-8')
    !gzip -f $fbienesinmuebles

    ## movemos los ficheros originales al directorio de procesados
    !mv $f11 $processed_dir
    !mv $f14 $processed_dir
    !mv $f15 $processed_dir
